In [ ]:
from dataclasses import dataclass
from io import StringIO
from pathlib import Path
from typing import Optional

import matplotlib.pylab as plt
import numpy as np
import pandas as pd
from pandas.testing import assert_frame_equal

import common_operations
import concentration_calculator
import data
import pipeline
import qc
import recovery
import utils

In [ ]:
# mock_peak_areas = "name,type,QC_1,QC_2,QC_3,QC_4,QC_5,ISRS_1,ISRS_2,ISRS_3,ISRS_4,ISRS_5\n13C_HCB,Area,20,40,60,80,100,20,40,60,80,100\nCB_207,Area,100,100,100,100,100,1000,1000,1000,1000,1000"
# mock_is_concentration_file = "name,amount\n13C_HCB,1000\nCB_207,1000"
# mock_sample_properties_file = "sample_name,sample_type,volume\nQC_1,qc,0.5\nQC_2,qc,0.5\nQC_3,qc,0.5\nQC_4,qc,0.5\nQC_5,qc,0.5\nISRS_1,isrs,0.5\nISRS_2,isrs,0.5\nISRS_3,isrs,0.5\nISRS_4,isrs,0.5\nISRS_5,isrs,0.5"
# mock_is_correspondence_file = (
#     "native,internal_standard,external_standard\nalpaHCH,13C_HCB,CB_207"
# )

In [ ]:
parent_folder = utils.Configuration.RAW_DATA_PATH

df = data.Data(
    quant_file=parent_folder.joinpath("results.csv"),
    is_correspondence_file=parent_folder.joinpath("is_std_table_correspondence.csv"),
    sample_properties_file=parent_folder.joinpath("sample_properties.csv"),
    qc_file=parent_folder.joinpath("qc.csv"),
    is_concentration_file=parent_folder.joinpath("is_std_table_concentration.csv"),
)

In [ ]:
data.DataValidator(df).validate_data()

In [ ]:
recovery.Recovery(df).plot_response_factor(by_sample=True)

In [ ]:
qc.CorrectionFactor(df).plot_correction_factor()

In [ ]:
concentration_calculator.ConcentrationCalculator(
    df, qc.CorrectionFactor(df).calculate_correction_factor()
).plot_concentration(by_sample=False)

In [ ]:
parent_folder = utils.Configuration.RAW_DATA_PATH

file_paths = {
    "quant_file": parent_folder.joinpath("results.csv"),
    "is_correspondence_file": parent_folder.joinpath("is_std_table_correspondence.csv"),
    "sample_properties_file": parent_folder.joinpath("sample_properties.csv"),
    "qc_file": parent_folder.joinpath("qc.csv"),
    "is_concentration_file": parent_folder.joinpath("is_std_table_concentration.csv"),
}

job = pipeline.Pipeline(**file_paths)

job.execute()

# Trubleshooting

In [ ]:
# TODO check what files are needed for the execution of recovery_calculator, correction_factor_calculator and
# concentration_calculator and what classes rely on what other ones.

parent_folder = utils.Configuration.RAW_DATA_PATH

file_paths = {
    "quant_file": parent_folder.joinpath("results.csv"),
    "is_correspondence_file": parent_folder.joinpath("is_std_table_correspondence.csv"),
    "sample_properties_file": parent_folder.joinpath("sample_properties.csv"),
    # "qc_file": parent_folder.joinpath("qc.csv"),
    # "is_concentration_file": parent_folder.joinpath("is_std_table_concentration.csv"),
}

dfs = data.Data(**file_paths)
data_validator = data.DataValidator(dfs)
recovery_calculator = recovery.Recovery(dfs)
correction_factor_calculator = qc.CorrectionFactor(dfs)
concentration_calc = concentration_calculator.ConcentrationCalculator(
    dfs, correction_factor_calculator.calculate_correction_factor()
)


job = pipeline.Pipeline(
    dfs,
    data_validator,
    recovery_calculator,
    correction_factor_calculator,
    concentration_calc,
)
job.execute()

In [ ]:
correction_factor_calculator.calculate_correction_factor()